In [1]:
import scanpy as sc
import anndata
import os
import fnmatch
%matplotlib inline
import matplotlib.pylab as plt
import pandas as pd
import numpy as np
#import scrublet as scr
import importlib
import sys
from harmony import harmonize
import seaborn as sns
from matplotlib import rcParams
import math
from scipy import stats
import seaborn as sns
import scipy as sp
import h5py
import warnings
import random
import math
import scvelo as scv
from matplotlib import rcParams
from matplotlib import cm
from matplotlib.colors import LinearSegmentedColormap

In [2]:
#Setting Scanpy figure parameters 
sc.settings.set_figure_params(dpi = 500, color_map = 'RdPu', dpi_save = 500)

# 1. Import .h5 files

In [4]:
import os
import fnmatch

directory = './CellBender_Filtered_h5_files_10March2022'
granuloma = '*Gran*'

matching_files = []

for filename in os.listdir(directory):
    if fnmatch.fnmatch(filename, granuloma):
        full_file_path = os.path.join(directory, filename)
        matching_files.append(full_file_path)

print(matching_files)
print(f"Number of matching files: {len(matching_files)}")


['/Users/joshuadavidbromley/Dropbox/Desktops/CellBender_Filtered_h5_files_10March2022/13518_Array4Granulo.h5', '/Users/joshuadavidbromley/Dropbox/Desktops/CellBender_Filtered_h5_files_10March2022/12818_Array4Granulo.h5', '/Users/joshuadavidbromley/Dropbox/Desktops/CellBender_Filtered_h5_files_10March2022/12518_Array4Granulo.h5', '/Users/joshuadavidbromley/Dropbox/Desktops/CellBender_Filtered_h5_files_10March2022/13918_Array11Granul.h5', '/Users/joshuadavidbromley/Dropbox/Desktops/CellBender_Filtered_h5_files_10March2022/13918_Array10Granul.h5', '/Users/joshuadavidbromley/Dropbox/Desktops/CellBender_Filtered_h5_files_10March2022/14018_Array3Granulo.h5', '/Users/joshuadavidbromley/Dropbox/Desktops/CellBender_Filtered_h5_files_10March2022/13818_Array9Granulo.h5', '/Users/joshuadavidbromley/Dropbox/Desktops/CellBender_Filtered_h5_files_10March2022/12518_Array2Granulo.h5', '/Users/joshuadavidbromley/Dropbox/Desktops/CellBender_Filtered_h5_files_10March2022/12818_Array2Granulo.h5', '/Users/j

In [5]:
import os

adata_list = []
for f in matching_files:
    base_filename = os.path.basename(f) 

    monkey = base_filename[0:5]  
    array = base_filename.split("_")[-1]  
    file = base_filename[-5:]  

    a = sc.read_10x_h5(f)
    a.var_names_make_unique()

    new_obs_names = [monkey + "_" + array + "_" + i for i in a.obs.index]
    a.obs_names = [i.replace('.h5', '') for i in new_obs_names]  
    adata_list.append(a)
    print(base_filename)  

13518_Array4Granulo.h5
12818_Array4Granulo.h5
12518_Array4Granulo.h5
13918_Array11Granul.h5
13918_Array10Granul.h5
14018_Array3Granulo.h5
13818_Array9Granulo.h5
12518_Array2Granulo.h5
12818_Array2Granulo.h5
13518_Array2Granulo.h5
14018_Array5Granulo.h5
13518_Array5Granulo.h5
12818_Array5Granulo.h5
12518_Array5Granulo.h5
13818_Array12Granul.h5
14018_Array2Granulo.h5
13818_Array8Granulo.h5
14218_Array11Granul.h5
14218_Array10Granul.h5
12518_Array3Granulo.h5
12818_Array3Granulo.h5
13518_Array3Granulo.h5
14018_Array4Granulo.h5
28618_Array4Granulo.h5
12818_Array6Granulo.h5
13518_Array6Granulo.h5
28318_Array3Granulo.h5
13918_Array12Granul.h5
14218_Array9Granulo.h5
13918_Array8Granulo.h5
28618_Array5Granulo.h5
14218_Array12Granul.h5
13818_Array11Granul.h5
13818_Array10Granul.h5
14218_Array8Granulo.h5
13918_Array9Granulo.h5


In [6]:
all_adata = sc.concat(adata_list,join='outer')
all_adata

AnnData object with n_obs × n_vars = 171974 × 32152

In [7]:
adata_allcells = sc.read_h5ad("./Scanpy_H5AD/adata_Final_for_Reinfection_publication.h5ad")
adata_allcells

AnnData object with n_obs × n_vars = 88360 × 25901
    obs: 'sample', 'DGE_name', 'monkey', 'array number', 'Sample Name', 'Sample type', 'Time point of sampling', 'Infusion before 2nd Mtb infection anti CD4 or IgG', 'Naïve or Primary Infection or Reinfection at sample time', 'Group', 'Group_Detailed', 'Drug Treatment', 'CFU Total ', 'Type of Mtb strain ', 'n_genes', 'n_counts', 'percent_mito', 'percent_ribo', 'percent_hbb', 'predicted_doublets', 'color', 'SubclusteringGroup', 'Subclustering', 'Doublet', 'leiden', 'leiden_lab_orig', 'Doublet_labels', 'Doublet_colors', 'Doublet_str', 'Doublet_int', 'CoarseClustering', 'SubclusteringV2'
    var: 'n_cells', 'Blacklist_Genes', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'CoarseClustering_colors', 'Doublet_colors_colors', 'Doublet_str_colors', 'Group_colors', 'SubclusteringV2_colors', 'Subclustering_colors', 'hvg', 'leiden', 'leiden_colors', 'leiden_lab_orig_colors', 'log1p', 'neighbors', 'pca', 'rank_genes_groups

In [8]:
cells_use = list(set(adata_allcells.obs_names).intersection(all_adata.obs_names))
all_adata= all_adata[cells_use]
all_adata.obs = adata_allcells.obs.loc[cells_use]
all_adata.var_names_make_unique()
all_adata

AnnData object with n_obs × n_vars = 88360 × 32152
    obs: 'sample', 'DGE_name', 'monkey', 'array number', 'Sample Name', 'Sample type', 'Time point of sampling', 'Infusion before 2nd Mtb infection anti CD4 or IgG', 'Naïve or Primary Infection or Reinfection at sample time', 'Group', 'Group_Detailed', 'Drug Treatment', 'CFU Total ', 'Type of Mtb strain ', 'n_genes', 'n_counts', 'percent_mito', 'percent_ribo', 'percent_hbb', 'predicted_doublets', 'color', 'SubclusteringGroup', 'Subclustering', 'Doublet', 'leiden', 'leiden_lab_orig', 'Doublet_labels', 'Doublet_colors', 'Doublet_str', 'Doublet_int', 'CoarseClustering', 'SubclusteringV2'

In [9]:
t=all_adata.X.toarray()
t

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [11]:
pd.DataFrame(data=t, index=all_adata.obs_names, columns=all_adata.var_names).T.to_csv("./Expression_Matrix_Reinfection_CD4_for_Multinichnet.csv")

In [10]:
pd.DataFrame(all_adata.obs[['sample', 'DGE_name', 'monkey', 'array number', 'Sample Name', 'Sample type', 'Group', 'Group_Detailed', 'Drug Treatment', 'n_genes', 'n_counts','SubclusteringV2',"CoarseClustering"]]).to_csv('./Metadata_Matrix_Reinfection_CD4_for_Multinichnet.csv')